<a href="https://colab.research.google.com/github/accsj/mvp-heart-failure/blob/main/mvp_heart_failure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVP - Predição de Doença Cardíaca
**Disciplina:** Engenharia de Sistemas de Software Inteligentes  
**Aluno:** Rodrigo Cunha Neto

## Objetivo
Este projeto busca treinar um modelo de machine learning capaz de prever se um paciente tem ou não doença cardíaca, utilizando dados clínicos. O processo inclui: leitura dos dados, pré-processamento, treinamento de modelos clássicos (KNN, Árvore de Decisão, Naive Bayes e SVM), validação, comparação de desempenho e exportação do melhor modelo.


In [ ]:
# Importando as bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score
import joblib


In [ ]:
# Carregando o arquivo heart.csv
import pandas as pd

df = pd.read_csv('heart.csv')
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [ ]:
# Verificando os dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [ ]:
# Checando os valores únicos de cada coluna categórica
categorical_columns = ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']
for col in categorical_columns:
    print(f'{col}: {df[col].unique()}')

Sex: ['M' 'F']
ChestPainType: ['ATA' 'NAP' 'ASY' 'TA']
RestingECG: ['Normal' 'ST' 'LVH']
ExerciseAngina: ['N' 'Y']
ST_Slope: ['Up' 'Flat' 'Down']


In [ ]:
# Separando as colunas de entrada (X) e o alvo (y)
X = df.drop('HeartDisease', axis=1)
y = df['HeartDisease']

In [ ]:
# Definindo quais colunas são numéricas e quais são categóricas
numeric_features = ['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak']
categorical_features = ['Sex', 'ChestPainType', 'FastingBS', 'RestingECG', 'ExerciseAngina', 'ST_Slope']


In [ ]:
# Criando o pré-processador

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# Pré-processador que normaliza os números e codifica as categorias
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])


In [ ]:
# Dividindo os dados entre treino e teste

from sklearn.model_selection import train_test_split

# Separando 80% para treino e 20% para teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# Pipeline KNN
pipeline_knn = Pipeline([
    ('preprocess', preprocessor),
    ('classifier', KNeighborsClassifier())
])

pipeline_knn.fit(X_train, y_train)

y_pred_knn = pipeline_knn.predict(X_test)
print("KNN")
print(classification_report(y_test, y_pred_knn))


KNN
              precision    recall  f1-score   support

           0       0.88      0.89      0.88        82
           1       0.91      0.90      0.91       102

    accuracy                           0.90       184
   macro avg       0.90      0.90      0.90       184
weighted avg       0.90      0.90      0.90       184



In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Pipeline Árvore de Classificação
pipeline_tree = Pipeline([
    ('preprocess', preprocessor),
    ('classifier', DecisionTreeClassifier(random_state=42))
])

pipeline_tree.fit(X_train, y_train)

y_pred_tree = pipeline_tree.predict(X_test)
print("Árvore de Classificação")
print(classification_report(y_test, y_pred_tree))


Árvore de Classificação
              precision    recall  f1-score   support

           0       0.78      0.76      0.77        82
           1       0.81      0.82      0.82       102

    accuracy                           0.79       184
   macro avg       0.79      0.79      0.79       184
weighted avg       0.79      0.79      0.79       184



In [ ]:
from sklearn.naive_bayes import GaussianNB

# Pipeline Naive Bayes

# Para usar o GaussianNB precisamos que os dados já estejam numéricos. Portanto teremos que transformar manualmente.
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

nb = GaussianNB()
nb.fit(X_train_transformed, y_train)
y_pred_nb = nb.predict(X_test_transformed)

print("Naive Bayes")
print(classification_report(y_test, y_pred_nb))


Naive Bayes
              precision    recall  f1-score   support

           0       0.88      0.84      0.86        82
           1       0.88      0.91      0.89       102

    accuracy                           0.88       184
   macro avg       0.88      0.88      0.88       184
weighted avg       0.88      0.88      0.88       184



In [ ]:
from sklearn.svm import SVC

# Pipeline SVM
pipeline_svm = Pipeline([
    ('preprocess', preprocessor),
    ('classifier', SVC(kernel='rbf', probability=True))
])

pipeline_svm.fit(X_train, y_train)

y_pred_svm = pipeline_svm.predict(X_test)
print("SVM")
print(classification_report(y_test, y_pred_svm))


SVM
              precision    recall  f1-score   support

           0       0.92      0.85      0.89        82
           1       0.89      0.94      0.91       102

    accuracy                           0.90       184
   macro avg       0.90      0.90      0.90       184
weighted avg       0.90      0.90      0.90       184



In [ ]:
# O modelo com maior assertividade foi o SVM, então iremos salvar o modelo como .pkl
import joblib

joblib.dump(pipeline_svm, 'modelo_svm_heart.pkl')
print("Modelo exportado com sucesso!")

Modelo exportado com sucesso!


Neste trabalho, construímos um pipeline completo de predição de doenças cardíacas a partir de dados clínicos. O processo envolveu desde a leitura e análise dos dados, passando por um pré-processamento cuidadoso com normalização de variáveis numéricas e codificação de variáveis categóricas, até a construção e avaliação de diferentes algoritmos de machine learning.

Os modelos testados foram: K-Nearest Neighbors (KNN), Árvore de Decisão, Naive Bayes e Support Vector Machine (SVM). Dentre eles, o modelo SVM apresentou o melhor desempenho, com uma acurácia de aproximadamente 90%, além de valores equilibrados de precisão e recall para ambas as classes (com ou sem doença). Isso demonstra que o modelo conseguiu aprender padrões relevantes e realizar uma boa generalização mesmo com um conjunto de dados moderado.

Como etapa final, o modelo SVM foi salvo no formato .pkl, permitindo sua reutilização em aplicações futuras, como uma interface web de predição em tempo real.

O principal aprendizado deste projeto foi entender na prática como os dados clínicos podem ser transformados em insights valiosos através da inteligência artificial. Além disso, reforçamos a importância do pré-processamento, da escolha adequada dos algoritmos e da análise cuidadosa dos resultados para garantir um modelo confiável.